## Day 35 Lecture 2 Assignment

In this assignment, we will combine what we have learned so far about classification algorithms this week.

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
admission = pd.read_csv('https://tf-assets-prod.s3.amazonaws.com/tf-curric/data-science/Admission_Predict.csv')

In [3]:
admission.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,324,107,4,4.0,4.5,8.87,1,0.76
2,3,316,104,3,3.0,3.5,8.00,1,0.72
3,4,322,110,3,3.5,2.5,8.67,1,0.80
4,5,314,103,2,2.0,3.0,8.21,0,0.65


In this assignment, we will predict the probability of a student being admitted to a PhD program given their stats. To make the predictions, find the median for the Chance of Admit column. Create an admit column where all probabilities above the median will receive a 1 for that column and all probabilities below the median will be a zero.

Below you will process and clean the data, try the SVM classifier, the gradient boosted decision tree classifier and XGBoost, and compare your results.

Have fun!

In [4]:
# Answer below:
admission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Serial No.         400 non-null    int64  
 1   GRE Score          400 non-null    int64  
 2   TOEFL Score        400 non-null    int64  
 3   University Rating  400 non-null    int64  
 4   SOP                400 non-null    float64
 5   LOR                400 non-null    float64
 6   CGPA               400 non-null    float64
 7   Research           400 non-null    int64  
 8   Chance of Admit    400 non-null    float64
dtypes: float64(4), int64(5)
memory usage: 28.2 KB


In [5]:
admission.describe()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,200.500000,316.807500,107.410000,3.087500,3.400000,3.452500,8.598925,0.547500,0.724350
std,115.614301,11.473646,6.069514,1.143728,1.006869,0.898478,0.596317,0.498362,0.142609
min,1.000000,290.000000,92.000000,1.000000,1.000000,1.000000,6.800000,0.000000,0.340000
25%,100.750000,308.000000,103.000000,2.000000,2.500000,3.000000,8.170000,0.000000,0.640000
50%,200.500000,317.000000,107.000000,3.000000,3.500000,3.500000,8.610000,1.000000,0.730000
75%,300.250000,325.000000,112.000000,4.000000,4.000000,4.000000,9.062500,1.000000,0.830000
max,400.000000,340.000000,120.000000,5.000000,5.000000,5.000000,9.920000,1.000000,0.970000


In [6]:
admission.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR ', 'CGPA', 'Research', 'Chance of Admit '],
      dtype='object')

In [13]:
new_columns = ['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit']

admission.columns = new_columns

admission.columns

Index(['Serial No.', 'GRE Score', 'TOEFL Score', 'University Rating', 'SOP',
       'LOR', 'CGPA', 'Research', 'Chance of Admit'],
      dtype='object')

In [15]:
admission['Chance of Admit'].median()

0.73

In [17]:
admission['Chance of Admit'] = np.where(admission['Chance of Admit'] >= 0.73,1,0)
admission['Chance of Admit'].value_counts()

1    204
0    196
Name: Chance of Admit, dtype: int64

In [18]:
from sklearn.model_selection import train_test_split

X = admission.drop('Chance of Admit', axis=1)
y = admission['Chance of Admit']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [20]:
#svm model
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from datetime import datetime

start_time = datetime.now()
svc = SVC(kernel='linear',C=10)

svc.fit(X_train, y_train)
end_time = datetime.now()

print("Train score: ",svc.score(X_train, y_train))
print("Test score: ",svc.score(X_test, y_test))
print('Duration: {}'.format(end_time - start_time))

Train score:  0.884375
Test score:  0.8625
Duration: 0:00:03.828979


In [55]:
start_time = datetime.now()
param_grid = {'kernel': ['linear','rbf','poly'], 'C':np.arange(1,100,3)}

svc = SVC()

svc_grid = GridSearchCV(svc, param_grid=param_grid, scoring='accuracy', cv=3)

svc_grid.fit(X_train, y_train)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:06:49.004888


In [56]:
print("Best kernal SVC hyperpararmeter: ",svc_grid.best_estimator_.kernel)
print("Best C SVC hyperpararmeter: ",svc_grid.best_estimator_.C)

Best kernal SVC hyperpararmeter:  linear
Best C SVC hyperpararmeter:  7


In [57]:
best_svc = SVC(kernel= 'linear', C=7)

best_svc.fit(X_train, y_train)

print("Best Hyperparameters SVC Train score: ",best_svc.score(X_train, y_train))
print("Best Hyperparameters SVC Test score: ",best_svc.score(X_test, y_test))

Best Hyperparameters SVC Train score:  0.884375
Best Hyperparameters SVC Test score:  0.8625


In [37]:
start_time = datetime.now()
gbc = GradientBoostingClassifier()

gbc.fit(X_train, y_train)
end_time = datetime.now()

print("Train score: ",gbc.score(X_train, y_train))
print("Test score: ",gbc.score(X_test, y_test))
print('Duration: {}'.format(end_time - start_time))

Train score:  0.99375
Test score:  0.875
Duration: 0:00:00.110876


In [60]:
start_time = datetime.now()
param_grid = {'learning_rate': np.arange(.01,.3,.01),'n_iter_no_change': [10],'n_estimators':[1000], 'max_depth': [2,3,], 
              'max_features': ['sqrt',.75,1]}

gbc = GradientBoostingClassifier()

gbc_grid = GridSearchCV(gbc, param_grid=param_grid, scoring='accuracy', cv=5)

gbc_grid.fit(X_train, y_train)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 0:00:37.699742


In [61]:
print("Best learning rate GBC hyperpararmeter: ",gbc_grid.best_estimator_.learning_rate)
print("Best max depth GBC hyperpararmeter: ",gbc_grid.best_estimator_.max_depth)
print("Best max features GBC hyperpararmeter: ",gbc_grid.best_estimator_.max_features)

Best learning rate GBC hyperpararmeter:  0.060000000000000005
Best max depth GBC hyperpararmeter:  2
Best max features GBC hyperpararmeter:  1


In [62]:
best_gbc = GradientBoostingClassifier(learning_rate=.06,n_iter_no_change=10,n_estimators=1000, max_depth=2, max_features=1)

best_gbc.fit(X_train, y_train)

print("Best Hyperparameters GBC Train score: ",best_gbc.score(X_train, y_train))
print("Best Hyperparameters GBC Test score: ",best_gbc.score(X_test, y_test))

Best Hyperparameters GBC Train score:  0.928125
Best Hyperparameters GBC Test score:  0.8875


In [63]:
from xgboost import XGBClassifier

start_time = datetime.now()
xgb = XGBClassifier()

xgb.fit(X_train, y_train)
end_time = datetime.now()

print("Train score: ",xgb.score(X_train, y_train))
print("Test score: ",xgb.score(X_test, y_test))
print('Duration: {}'.format(end_time - start_time))

Train score:  0.971875
Test score:  0.8875
Duration: 0:00:00.031829


In [64]:
start_time = datetime.now()
param_grid = {'subsample': np.arange(.5,1,.1), 'learning_rate': np.arange(.01,.3,.01), 'max_depth': np.arange(3,10,1), 
              'lambda': np.arange(0,1,.1),'max_features': ['sqrt',.75,1]}

xgb = XGBClassifier()

xgb_grid = GridSearchCV(xgb, param_grid=param_grid, scoring='accuracy', cv=5)

xgb_grid.fit(X_train, y_train)
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

Duration: 1:17:20.141225


In [70]:
print("Best learning rate GBC hyperpararmeter: ",gbc_grid.best_estimator_.learning_rate)
print("Best max depth GBC hyperpararmeter: ",gbc_grid.best_estimator_.max_depth)
print("Best max features GBC hyperpararmeter: ",gbc_grid.best_estimator_.max_features)
print("Best alpha features GBC hyperpararmeter: ",gbc_grid.best_estimator_.alpha)
print("Best sub sample GBC hyperpararmeter: ",gbc_grid.best_estimator_.subsample)

Best learning rate GBC hyperpararmeter:  0.060000000000000005
Best max depth GBC hyperpararmeter:  2
Best max features GBC hyperpararmeter:  1
Best alpha features GBC hyperpararmeter:  0.9
Best sub sample GBC hyperpararmeter:  1.0


In [71]:
start_time = datetime.now()
xgb = XGBClassifier(learning_rate=.06,max_depth=2,max_features=1, alpha=.9, subsample=1)

xgb.fit(X_train, y_train)
end_time = datetime.now()

print("Train score: ",xgb.score(X_train, y_train))
print("Test score: ",xgb.score(X_test, y_test))
print('Duration: {}'.format(end_time - start_time))

Train score:  0.91875
Test score:  0.9
Duration: 0:00:00.028604


The XGBoost model had the best optimized test accuray of any model. It also did the least overfitting. 